In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from astropy import units as u
import glob
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord
from pylab import *
from astropy.coordinates import match_coordinates_sky
from itertools import cycle
import scipy.integrate as integrate
from scipy.interpolate import griddata
from itertools import permutations
from itertools import combinations
from collections import Counter
from itertools import product
from math import e
from matplotlib import transforms
import matplotlib.colors as colors

import scipy
from scipy import io
from scipy.stats import iqr, norm
from scipy.stats.kde import gaussian_kde
from astropy.io import fits

from astropy.time import Time
from astropy import wcs
from astropy.coordinates import SkyCoord, EarthLocation, Angle, FK5

import skimage
import scipy as sp
from skimage.feature import register_translation 
from astropy.io import fits

from matplotlib import pyplot, transforms
from scipy import ndimage

from matplotlib import cm, colors

import cartopy.crs as ccrs

In [ ]:
#format the matplotlib graphs
plt.rcParams['figure.figsize'] = (10, 10)
plt.rc('axes', labelsize=14)
plt.rc('axes', labelweight='bold')
plt.rc('axes', titlesize=16)
plt.rc('axes', titleweight='bold')
plt.rc('font', family='sans-serif')

In [ ]:
def collector(path):
    '''
    The collector reads one or multiple IDL .sav files
    and appends each .sav file's entire data structure to a list.
    
    Args:
        path: The file system location of an IDL .sav file
            or a folder of IDL .sav files.

    Returns:
        data: A dictonary containing two keys:
            1. "data": The data structure of the IDL .sav file(s).
            2. "filenames": The filename associated with the IDL .sav file(s).
        
    '''

    # Glob module finds all the pathnames matching a specified pattern
    # according to the rules used by the Unix shell.
    filenames = glob.glob(path)

    # Filenames are read into python with the scipy.io.readsav function.
    # Data structures and filenames and appended to their own lists
    # with dictionary keys.
    data = {'data': [scipy.io.readsav(filenames[i], python_dict=True)
            for i in range(len(filenames))], 'filenames': filenames}

    # An example for accessing the data structure from the
    # first or only .sav file read in by collector:
    # data[0]['data']
    #
    # An example for accessing the filename from the
    # fourth .sav file read in by collector:
    # data[3]['filenames']

    return data


In [ ]:
directory = '/Users/student/kelcey/new source arrays/'
paths = glob.glob(directory + '*.sav')
GLEAM_path = '/Users/student/kelcey/GLEAM_v2_plus_rlb2019.sav'
GLEAM_data = collector(GLEAM_path)

In [ ]:
def match_to_gleam(directory, GLEAM_data, distance_restriction, magnitude_restriction):
    """
    Matches all the .sav files in a given directory with GLEAM. This function can handle data
    stored under either 'catalog' or 'source_array' headers. A recomended number of observations
    would not exceed 30
    
    Arguments
        directory is an absolute path to the set of observations that end in 'source_array.sav'
        
        GLEAM_data is the output of the collector function after given an absolute path to the GLEAM catalog
        
        distance_restriction is the maximum distance in degrees away from the GLEAM object that the code
        will consider a match. This should be a decimal value with no units. Recommended value: 0.1
        
        magnitude_restriction is the restriction on the deviation from the GLEAM magnitude that the 
        code will consider a match. Values should be in fractional, decimal, or integer form. A match will
        be considered within the GLEAM magnitude +/- magnitude_restriction * GLEAM magnitude. Recommended
        value: 3/4
        
    Returns
        a pandas DataFrame with the generated data. The row names are the index of an object in GLEAM.The 
        column names are:
            GLEAM Columns
            
            RA: The RA recorded in GLEAM in degrees
            
            DEC: The DEC recorded in GLEAM in degrees
            
            Mag GLEAM: The magnitude recorded in GLEAM in Jy
            
            RA EO GLEAM: The extended RA components recorded in GLEAM in degrees
            
            DEC EO GLEAM: The extended DEC components recorded in GLEAM in degrees
            
            Mag EO GLEAM: The magnitude in Jy associated with each extended GLEAM component
            
            Individual Observation Columns
            
            Mag (Obesrvation Number): The higher level (single point) magnitude associated with 
            this observation in Jy. If the object is determined not to have a match in this observation set,
            a 0 is recorded. This is used to quickly index through the table and allows for other information
            about the observation to remain
            
            Distance (Observation Number):This represents the distance in this observation from the GLEAM
            object to its closest match. If this falls outside the specified parameters, a 0 will appear in 
            the Mag (Observation Number) column. The distance is in degrees.
            
            RA(Observation): This is the higher level (single point) value for the observed RA of this
            source in degrees.
            
            DEC(Observation): This is the higher level (single point) value for the observed DEC of this
            source in degrees. 
            
            EO RA (Observation): This is the RA in degrees for the array of extended components in the 
            observation. If there are not extended components recorded, the higher level value will be
            reported here.
    
    """
    #Create paths to all the .sav files in the specified directory
    paths = glob.glob(directory + '*source_array.sav')
    ra_gleam = []
    dec_gleam = []
    imag_gleam = []
    
    ra_gleam_ext = []
    dec_gleam_ext = []
    imag_gleam_ext = []
    #Load the GLEAM data
    size = GLEAM_data['data'][0]['catalog'].shape[0]
    for i in range(size):
        if GLEAM_data['data'][0]['catalog'][i]['EXTEND'] is None:
            ra_gleam.append(GLEAM_data['data'][0]['catalog'][i]['RA'])
            dec_gleam.append(GLEAM_data['data'][0]['catalog'][i]['DEC'])
            imag_gleam.append(GLEAM_data['data'][0]['catalog'][i]['FLUX']['I'])
            ra_gleam_ext.append(0)
            dec_gleam_ext.append(0)
            imag_gleam_ext.append(0)
        else:
            ra_gleam.append(GLEAM_data['data'][0]['catalog'][i]['RA'])
            dec_gleam.append(GLEAM_data['data'][0]['catalog'][i]['DEC'])
            imag_gleam.append(GLEAM_data['data'][0]['catalog'][i]['FLUX']['I'])
            glm_mag_eo = []
            for j in range(0, GLEAM_data['data'][0]['catalog'][i]['FLUX'].shape[0]):
                glm_mag_eo.append(GLEAM_data['data'][0]['catalog'][i]['EXTEND']['FLUX'][j]['I'])
            imag_gleam_ext.append(glm_mag_eo)
            ra_gleam_ext.append(GLEAM_data['data'][0]['catalog'][i]['EXTEND']['RA'])
            dec_gleam_ext.append(GLEAM_data['data'][0]['catalog'][i]['EXTEND']['DEC'])
                
            
            
    
    #Create a Pandas Data Frame with the RA, DEC, and GLEAM Magnitudes
    n=0
    df = pd.DataFrame({'RA': ra_gleam,'Mag GLEAM': imag_gleam,  'DEC' : dec_gleam, 'RA EO GLEAM':ra_gleam_ext, 
                      'DEC EO GLEAM': dec_gleam_ext, 'Mag EO GLEAM': imag_gleam_ext})
    
    #Look at each path in the directory
    #WARNING: 2 files takes 15-20 minutes to load. DO NOT change the indecies unless 
    #you are prepared to wait a while. There are over 300k stars in GLEAM
    for path in paths:
        
        #Collect the data for each path
        n = n + 1
        data = collector(path)
        #x = seeker_catalog(data, source_label = 'source_array')
        
        eo = []
        eo_ra = []
        eo_dec = []
        ps_RA = []
        ps_DEC = []
        i_mag = []
        EO_imag = []
        
        ps_ston = []
        eo_ston = []

        try:
            d_s = data['data'][0]['catalog']
        except:
            d_s = data['data'][0]['source_array']
     
        for d in d_s:
            if d['EXTEND'] is None:
                ps_RA.append(d['RA'])
                ps_DEC.append(d['DEC'])
                EO_imag.append(d['FLUX']['I'])
                eo_ra.append(d['RA'])
                eo_dec.append(d['DEC'])
                i_mag.append(d['FLUX']['I'])
                ps_ston.append(d['STON'])
                eo_ston.append(d['STON'])
                
            else:
                ps_RA.append(d['RA'])
                ps_DEC.append(d['DEC'])
                EOmags = []
                eoston = []
                for i in range(0, d['EXTEND']['FLUX'].shape[0]):
                    EOmags.append(d['EXTEND']['FLUX'][i]['I'])
                    eoston.append(d['EXTEND']['STON'][i])
                eo_ston.append(np.array(eoston))
                EO_imag.append(np.array(EOmags))
                eo_ra.append(d['EXTEND']['RA'])
                eo_dec.append(d['EXTEND']['DEC'])
                i_mag.append(d['FLUX']['I'])
                ps_ston.append(d['STON'])

                
        #Match this path with the GLEAM catalog
        #idx: an array of indices corresponding to matches
        #d2d: the two dimensional distances between these matches
        #d3d: three dimensional distances between matches. This array is blank becasue we do 
        #not have 3 dimensional data, but the match_to_catalog_sky function requires it anyway
        catalog = SkyCoord(ra=ps_RA*u.deg, dec=ps_DEC*u.deg)  
        c = SkyCoord(ra=ra_gleam*u.deg, dec=dec_gleam*u.deg)  
        idx, d2d, d3d = c.match_to_catalog_sky(catalog)  
               
        #Possible source of error because the below statement is worded a little strange
        #Only return matches within specified parameters
        mags = []
        imags = np.array(i_mag)[idx]
        nums = np.arange(0, len(idx))
        for num in nums:
            if (d2d[num] < (distance_restriction*u.deg)) and ((imags[num] > imag_gleam[num] + (magnitude_restriction)*imag_gleam[num]) or (imags[num] > (magnitude_restriction)*imag_gleam[num])):
                mags.append(imags[num])
            else: 
                mags.append(0)

        #Add a new column to the data frame with the information from this path      
        s_mag = pd.Series(mags)
        s_dist = pd.Series(d2d)
        df['Mag {}'.format(n)] = s_mag
        df['Distance {}'.format(n)] = s_dist
        
        df['RA {}'.format(n)] = np.array(ps_RA)[idx]
        df['DEC {}'.format(n)] = np.array(ps_DEC)[idx]
        df['EO RA {}'.format(n)] = np.array(pd.Series(eo_ra))[idx]
        df['EO DEC {}'.format(n)] = np.array(pd.Series(eo_dec))[idx]
        df['EO Mag {}'.format(n)] = np.array(pd.Series(EO_imag))[idx]
        df['STON {}'.format(n)] = np.array(ps_ston)[idx]
        df['EO STON {}'.format(n)] = np.array(pd.Series(eo_ston))[idx]
        
    return df

In [ ]:
def plot_source(table,index ,xstretch, ystretch ):
    """
    Plots a source over many observations given its index in GLEAM, by default centered on the GLEAM object
    
    Arguments
        table is a pandas DataFrame generated by the match_to_gleam function
        
        index is the position of the desired object in table
        
        xstretch is the distance in degrees from the center of the plot to the edge in the x directio.
        A recomended value is 0.25
        
        ystretch is the distance in degrees from the center of the plot to the edge in the y direction.
        A recomended value is 0.25.
    Returns
        A saved .png in the directory running the code named 'Source_index_scatter'
    """
    #mags are the higher level magnitudes in Jy 
    mags = table.loc[index][6::9]
    #mags_eo are the extended magnitudes in Jy
    mags_eo = table.loc[index][12::9]
    #ras are the extended RA components in degrees
    ras = np.array(table.loc[index][10::9])
    #decs are the extended DEC components in degrees
    decs = np.array(table.loc[index][11::9])
    #gleam_mag is the magnitude for this object in GLEAM
    gleam_mag = table.loc[index]['Mag GLEAM']

    #A match in the table is indicated by a non zero number for the higher level magnitude. 
    #The following loops through these, selecting only the entries with non zero magnitudes
    for i in range(0, len(mags)):
        if mags[i] !=0:
            
            #For each match, a scatter plot of the components is made. 
            plt.scatter(ras[i], decs[i], 
                        label = 'Observation {}, {} Jy'.format(i+1, mags[i]))
            plt.legend();
    #GLEAM is plotted over these observations, displaying both the match at the center of the image and 
    #all other sources in this area 
    plt.scatter(table.loc[index]['RA'], table.loc[index]['DEC'], color = 'k', marker = '*',
                label = 'GLEAM, {} Jy'.format(gleam_mag))
    #Label the axes
    plt.xlabel('RA (Degrees)')
    plt.ylabel('DEC (Degrees)')
    #Generate the title
    plt.title('Source {}'.format(index))
    
    #Get the RA and DEC for this GLEAM object
    g_ra = table.loc[index]['RA']
    g_dec = table.loc[index]['DEC']
    
    #Format the graph with the defined stretch
    plt.xlim(g_ra-xstretch, g_ra+xstretch)
    plt.ylim(g_dec-ystretch, g_dec+ystretch)
 
    plt.xlim(g_ra-xstretch, g_ra+xstretch)
    plt.ylim(g_dec-ystretch, g_dec+ystretch)
    
    #display the legend
    plt.legend();
    
    #save the image
    plt.savefig('Source_{}_scatter'.png)

In [ ]:
def plot_source_proportional(table,index ,xstretch, ystretch ):
    """
    Plots a source over many observations given its index in GLEAM, by default centered on the GLEAM object.
    This code is identical to the plot_source function but displays the individual points in each
    observation with size proportional to their magnitudes in Jy
    
    Arguments
        table is a pandas DataFrame generated by the match_to_gleam function
        
        index is the position of the desired object in table
        
        xstretch is the distance in degrees from the center of the plot to the edge in the x directio.
        A recomended value is 0.25
        
        ystretch is the distance in degrees from the center of the plot to the edge in the y direction.
        A recomended value is 0.25.
    Returns
        A saved .png in the directory running the code named 'Source_index_scatter_proportional_markers'
    """
    #mags are the higher level magnitudes in Jy 
    mags = table.loc[index][6::9]
    #mags_eo are the extended magnitudes in Jy
    mags_eo = table.loc[index][12::9]
    #ras are the extended RA components in degrees
    ras = np.array(table.loc[index][10::9])
    #decs are the extended DEC components in degrees
    decs = np.array(table.loc[index][11::9])
    #gleam_mag is the magnitude for this object in GLEAM
    gleam_mag = table.loc[index]['Mag GLEAM']

    #A match in the table is indicated by a non zero number for the higher level magnitude. 
    #The following loops through these, selecting only the entries with non zero magnitudes
    for i in range(0, len(mags)):
        if mags[i] !=0:
            
            #For each match, a scatter plot of the components is made. 
            plt.scatter(ras[i], decs[i], s = mags_eo[i], 
                        label = 'Observation {}, {} Jy'.format(i+1, mags[i]))
            plt.legend();
    #GLEAM is plotted over these observations, displaying both the match at the center of the image and 
    #all other sources in this area 
    plt.scatter(table.loc[index]['RA'], table.loc[index]['DEC'], color = 'k', marker = '*',
                label = 'GLEAM, {} Jy'.format(gleam_mag))
    #Label the axes
    plt.xlabel('RA (Degrees)')
    plt.ylabel('DEC (Degrees)')
    #Generate the title
    plt.title('Source {}'.format(index))
    
    #Get the RA and DEC for this GLEAM object
    g_ra = table.loc[index]['RA']
    g_dec = table.loc[index]['DEC']
    
    #Format the graph with the defined stretch
    plt.xlim(g_ra-xstretch, g_ra+xstretch)
    plt.ylim(g_dec-ystretch, g_dec+ystretch)
 
    plt.xlim(g_ra-xstretch, g_ra+xstretch)
    plt.ylim(g_dec-ystretch, g_dec+ystretch)
    
    #display the legend
    plt.legend();
    
    #save the image
    plt.savefig('Source_{}_scatter_proportional_markers'.png)

In [ ]:
def shift_arrays(table, index):
    mags = table.loc[index][3::9]
    mags_eo = table.loc[index][9::9]
    mag_change = []
    ra_center = table.loc[index][5::9]
    dec_center = table.loc[index][6::9]
    ext_ras = table.loc[index][7::9]
    ext_decs = table.loc[index][8::9]
    ras = []
    decs = []
    cent_r = []
    adj_obs = []
    
    imnum = []
    
    true_eo_mags = []
    num = 0
    for i in range(0, len(mags)):
        num = num+1
        if mags[i] !=0:
            mag_change.append(mags[i])
            ras.append(ra_center[i])
            decs.append(dec_center[i])
            imnum.append(num)
    x_center = np.mean(ras)
    y_center = np.mean(decs)
    for i in range(0, len(mags)):
        if mags[i] !=0:
            rs = ext_ras[i]
            ds = ext_decs[i]
            ra_roll = x_center - ra_center[i]
            dec_roll = y_center - dec_center[i]
            new_array = rs+ra_roll, ds+dec_roll
            cent_r.append(x_center - ra_center[i])
            adj_obs.append(new_array)
            true_eo_mags.append(mags_eo[i])
            

        
    for r in range(len(adj_obs)):
        plt.scatter(adj_obs[r][0], adj_obs[r][1], #s = true_eo_mags[r], 
            label = 'Observation {}, {} Jy'.format(imnum[r], mag_change[r]))
    plt.xlabel('RA (Degrees)')
    plt.ylabel('DEC (Degrees)')
    plt.title('Source {}, Center Calculation'.format(index))
    
    g_ra = table.loc[index]['RA']
    g_dec = table.loc[index]['DEC']
    

    plt.scatter(table['RA'], table['DEC'],  marker = '*', color = 'k', label = 'GLEAM')
    
    plt.xlim(g_ra-.25, g_ra+.25)
    plt.ylim(g_dec-.25, g_dec+.25)
    
    plt.grid()
    
    plt.gca().invert_xaxis()
    
    plt.legend();
  

In [1]:
def gaussian(sigma, x_0, y_0, x, y, power):
    xs = (x-x_0)**2
    ys = (y-y_0)**2
    gauss_val = power * (e** ((-(xs + ys))/(2*(sigma**2))))
    return gauss_val